# AI Resume Analyzer — Demo Notebook

Run these cells to demo the project.

In [1]:
!pip install --upgrade pip setuptools wheel


In [3]:
!pip install pdfplumber==0.9.0
!pip install PyPDF2==3.0.1
!pip install spacy==3.6.0 --use-pep517
!pip install scikit-learn==1.3.0
!pip install pandas==2.1.0
!pip install numpy==1.26.0
!pip install regex
!pip install nbformat

  Using cached pdfplumber-0.9.0-py3-none-any.whl.metadata (35 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 13.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [pdfplumber]
  Using cached pypdf2-3.0.1-py3-none-any.whl.metadata (6.8 kB)
  Using cached spacy-3.6.0.tar.gz (1.3 MB)
  error: subprocess-exited-with-error
  
  × installing build dependencies for spacy did not run successfully.
  │ exit code: 1
  ╰─> No available output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Installing build dependencies ... error
ERROR: Failed to build 'spacy' when installing build dependencies for spacy
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 18.9 MB/s  0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-learn: filename=scikit_learn-1.3.0-cp312-cp312-linux_x86_64.whl size=10683878 sha256=6605646

In [10]:
from parser import parse_resume
from matcher import extract_keywords_from_text, match_skills, jd_similarity_score
from scoring import calculate_score, formatting_score
from feedback import generate_feedback

parsed = parse_resume('Samples/resume3.txt')
with open('Samples/jd_software_engineer.txt','r',encoding='utf-8') as f:
    jd_text = f.read()

jd_keywords = extract_keywords_from_text(jd_text, top_k=60)
matched, missing = match_skills(parsed.get('skills', []), jd_keywords)
skill_pct = len(matched)/len(jd_keywords) if jd_keywords else 0
keyword_sim = jd_similarity_score(parsed.get('raw_text',''), jd_text)
exp_score = 0.6 if parsed.get('experience') else 0.2
education_match = 1 if parsed.get('education') else 0
fmt = formatting_score(parsed.get('raw_text',''))
match_counts = {'skill_pct': skill_pct, 'keyword_pct': keyword_sim, 'exp_score': exp_score, 'education_match': education_match, 'formatting_score': fmt}
final_score = calculate_score(match_counts, {})
fb = generate_feedback(parsed, matched, missing, jd_keywords)

print('ATS Score:', final_score)
print('Matched:', matched)
print('Missing (top 10):', missing[:10])
print('Suggestions:', fb['suggestions'])


ATS Score: 14.63
Matched: ['john doe email: john.doe@example.com phone: +1 555 555 5555 summary frontend developer focused on react and modern javascript. experience frontend engineer', 'webcorp (2019-2024) - built spas using react', 'redux - improved accessibility and performance skills javascript', 'webpack', 'git']
Missing (top 10): ['agile', 'api', 'apis', 'aws', 'backend', 'build', 'cloud', 'collaborate', 'containerization', 'database']
Suggestions: ['Add or emphasize these skills/keywords: agile, api, apis, aws, backend, build, cloud, collaborate, containerization, database.', 'Make the top 3-4 lines a tailored summary that includes the role + 2-3 top skills from the JD.', 'Expand bullet points under each job to include metrics and impact (e.g., reduced latency by 30%).', 'If you have relevant certifications (e.g., AWS, GCP), list them in a Certifications section.', 'Include important JD keywords such as: agile, api, apis, aws, backend, build, cloud, collaborate.']
